In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import os

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
    
device = torch.device('cuda' if train_on_gpu else 'cpu')

CUDA is available!  Training on GPU ...


In [2]:
def load_X(path):
    X_signals = []
    files = os.listdir(path)
    files.sort(key=str.lower)
    #['train_acc_x.txt', 'train_acc_y.txt', 'train_acc_z.txt', 'train_gyr_x.txt', 'train_gyr_y.txt', 'train_gyr_z.txt']
    for my_file in files:
        fileName = os.path.join(path,my_file)
        file = open(fileName, 'r')
        X_signals.append(
            [np.array(cell, dtype=np.float32) for cell in [
                row.strip().split(' ') for row in file
            ]]
        )
        file.close()
        #X_signals = 6*totalStepNum*128
    X_signals = np.transpose(np.array(X_signals), (1, 0, 2))#(totalStepNum*6*128)
    return X_signals.reshape(-1,1,128,6)#(totalStepNum*6*128*1)

def load_y(y_path):
    file = open(y_path, 'r')
    # Read dataset from disk, dealing with text file's syntax
    y_ = np.array(
        [elem for elem in [
            row.replace('  ', ' ').strip().split(' ') for row in file
        ]],
        dtype=np.int32
    )
    file.close()
    return y_ - 1

    # Use this part only if you need to convert to 1-hot encoding

    # Substract 1 to each output class for friendly 0-based indexing
#     y_ = y_ - 1
    #one_hot
#     y_ = y_.reshape(len(y_))
#     n_values = int(np.max(y_)) + 1
#     return np.eye(n_values)[np.array(y_, dtype=np.int32)]  # Returns FLOATS

In [3]:
train_data_path = "..//test//train//x_train"
train_label_path = "..//test//train//y_train.txt"
test_data_path = "..//test//test//x_test"
test_label_path = "..//test//test//y_test.txt"

train_x = load_X(train_data_path)
train_y = load_y(train_label_path)
test_x  = load_X(test_data_path)
test_y  = load_y(test_label_path)

In [4]:
#create Tensor datasets
train_data = TensorDataset(torch.from_numpy(train_x), torch.from_numpy(train_y))
test_data  = TensorDataset(torch.from_numpy(test_x), torch.from_numpy(test_y))

#dataloaders
batch_size = 512
train_loader = DataLoader(train_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [47]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, kernel_size=(9,1), stride=(2,1), padding=(4,0))
        self.conv2 = nn.Conv2d(32,64, kernel_size=(3,1), padding=(1,0))
        self.conv3 = nn.Conv2d(64,128,kernel_size=(3,1), padding=(1,0))
        self.conv4 = nn.Conv2d(128,128,kernel_size=(1,6))
    
        self.pool = nn.MaxPool2d((2,1), stride=(2,1))
        
        self.fc1 = nn.Linear(128*16, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256,128)
        self.fc5 = nn.Linear(128, 20)
        
        
    def forward(self, x):
        
        # Convolution
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = self.conv4(x)
        
        # Flatten
        x = x.view(x.shape[0], -1)
        
        # Fully Connected
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
        return x
        
model = Net()
model.to(device)
print(model)

Net(
  (conv1): Conv2d(1, 32, kernel_size=(9, 1), stride=(2, 1), padding=(4, 0))
  (conv2): Conv2d(32, 64, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
  (conv3): Conv2d(64, 128, kernel_size=(3, 1), stride=(1, 1), padding=(1, 0))
  (conv4): Conv2d(128, 128, kernel_size=(1, 6), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2048, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=128, bias=True)
  (fc5): Linear(in_features=128, out_features=20, bias=True)
)


In [48]:
# Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [49]:
#Train

epochs = 10


for e in range(1, epochs+1):
    train_loss , test_loss = 0, 0
    train_accuracy, test_accuracy = 0, 0
    
    model.train()
    
    for data, labels in train_loader:
        data = data.to(device)
        labels = labels.to(device, dtype=torch.int64).view(labels.shape[0])
        
        optimizer.zero_grad()
        
        output = model(data)
        
        loss = criterion(output, labels)
        
        loss.backward()
        
        optimizer.step()
        
        train_loss += loss.item()
        
        top_p, top_class = output.topk(1, dim=1)
        equals = top_class == labels.view(*top_class.shape)
        train_accuracy += torch.mean(equals.to(dtype=torch.float))
        
    else:
        model.eval()
        for data, labels in test_loader:
            data = data.to(device)
            labels = labels.to(device, dtype=torch.int64).view(labels.shape[0])
            
            output = model(data)
            loss = criterion(output, labels)
            
            test_loss += loss.item()
            
            top_p, top_class = output.topk(1, dim=1)
            equals = top_class == labels.view(*top_class.shape)
            test_accuracy += torch.mean(equals.to(dtype=torch.float))
            
        print('Epoch %d,  Train Loss: %.5f,  Test Loss: %.5f,  Train Accuracy %.2f,  Test Accuracy %.2f' %
              (e, train_loss/len(train_loader), test_loss/len(test_loader), 
               train_accuracy/len(train_loader), test_accuracy/len(test_loader)))

Epoch 1,  Train Loss: 1.07072,  Test Loss: 0.87261,  Train Accuracy 0.67,  Test Accuracy 0.87
Epoch 2,  Train Loss: 0.20833,  Test Loss: 0.63102,  Train Accuracy 0.94,  Test Accuracy 0.91
Epoch 3,  Train Loss: 0.13515,  Test Loss: 0.67079,  Train Accuracy 0.96,  Test Accuracy 0.91
Epoch 4,  Train Loss: 0.09728,  Test Loss: 0.55673,  Train Accuracy 0.97,  Test Accuracy 0.93
Epoch 5,  Train Loss: 0.07962,  Test Loss: 0.57667,  Train Accuracy 0.98,  Test Accuracy 0.93


KeyboardInterrupt: 

In [50]:
torch.save(model.state_dict(), 'identify_cnn.pth')

In [ ]:
# To load in the pretrained weights
#state_dict = torch.load('identify_cnn.pth')
#model.load_state_dict(state_dict)